<a href="https://colab.research.google.com/github/poojasaxena/tensorflow-developer-zertificate-coursera/blob/main/course2_convolutional-neural-networks-tensorflow/06_saving_resuing_custom_model/Course_2_Part_1_Lesson_5_Notebook_Loading_weights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%autosave 60

Autosaving every 60 seconds


## 1. Setup

In [2]:
import numpy as np
import tensorflow as tf
import keras
print(tf.__version__)

2.5.0


In [8]:
! pip3 install --user pydot #pydotplus #graphviz

## 2. Saving Weights
You can choose to only save & load a model's weights. This can be useful if:

1. You only need the model for inference: in this case you won't need to restart training, so you don't need the compilation information or optimizer state.
2. You are doing transfer learning: in this case you will be training a new model reusing the state of a prior model, so you don't need the compilation information of the prior model.

**APIs for in-memory weight transfer**

Weights can be copied between different objects by using get_weights and set_weights:

1. tf.keras.layers.Layer.get_weights(): Returns a list of numpy arrays.
2. tf.keras.layers.Layer.set_weights(): Sets the model weights to the values in the weights argument.



### 2.1 Transfering weights from one layer to another, in memory



In [ ]:
def create_layer():
    layer = keras.layers.Dense(64, activation='relu', name='dense_2')
    layer.build((None, 784))
    return layer

In [ ]:
layer_1 = create_layer()
layer_2 = create_layer()
layer_1.get_weights()

[array([[ 0.06705026,  0.0302926 , -0.00659138, ...,  0.08303652,
         -0.01864984, -0.08171301],
        [-0.01530679, -0.0321387 , -0.01983957, ...,  0.06687555,
          0.06886926, -0.05028033],
        [ 0.06656461, -0.0038603 ,  0.0155442 , ..., -0.051704  ,
         -0.05591897,  0.05771992],
        ...,
        [ 0.02942144,  0.07910962,  0.05550406, ..., -0.05196543,
          0.07128374, -0.00816881],
        [ 0.04229943,  0.0442199 ,  0.08373808, ...,  0.0478973 ,
          0.02713925, -0.00378484],
        [-0.06299121, -0.00890368, -0.03250149, ..., -0.06255534,
          0.03694487,  0.00574443]], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)]

In [ ]:
# copy weights from layer_1 to layer_2
layer_2.set_weights(layer_1.get_weights())

In [ ]:
layer_2.get_weights()

[array([[ 0.06705026,  0.0302926 , -0.00659138, ...,  0.08303652,
         -0.01864984, -0.08171301],
        [-0.01530679, -0.0321387 , -0.01983957, ...,  0.06687555,
          0.06886926, -0.05028033],
        [ 0.06656461, -0.0038603 ,  0.0155442 , ..., -0.051704  ,
         -0.05591897,  0.05771992],
        ...,
        [ 0.02942144,  0.07910962,  0.05550406, ..., -0.05196543,
          0.07128374, -0.00816881],
        [ 0.04229943,  0.0442199 ,  0.08373808, ...,  0.0478973 ,
          0.02713925, -0.00378484],
        [-0.06299121, -0.00890368, -0.03250149, ..., -0.06255534,
          0.03694487,  0.00574443]], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)]

### 2.2 Transfering weights from one model to another model with a compatible architecture, in memory



In [3]:
## create a simple functional model
inputs = keras.Input(shape=(784, ), name='digits')
x = keras.layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = keras.layers.Dense(64, activation='relu', name='dense_2')(x)
outputs= keras.layers.Dense(10, activation='softmax', name='prediction')(x)
functional_model = keras.Model(inputs, outputs, name="3_layer_mlp" )

In [10]:
!apt-get -qq install -y graphviz && pip install pydot
import pydot

In [11]:
from keras.utils import plot_model

ImportError: ignored

In [13]:
keras.utils.plot_model(functional_model, "mini_model.png", show_shapes=True)


AttributeError: ignored

In [ ]:
## Create a subclass model with the same architecture
class SubClassModel(keras.Model):
    def __init__(self, output_dim, name='None'):
        super(SubClassModel, self).__init__(name=name)
        self.output_dim=output_dim
        self.dense_1 = keras.layers.Dense(64, activation='relu', name='dense_1')
        self.dense_2 = keras.layers.Dense(64, activation='relu', name='dense_2')
        self.dense_3 = keras.layers.Dense(output_dim, activation='softmax', name='prediction')
        
    def call(self, inputs):
        x = self.dense_1(inputs)
        x = self.dense_2(x)
        x = self.dense_3(x)
        return x
    
    def get_config(self):
        return {"output_dim":self.output_dim, "name":self.name}

In [ ]:
subclassmodel = SubClassModel(10)

In [ ]:
## call the SubclassModel once again to create the weights
subclassmodel(tf.ones((1, 784)))

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[0.03246441, 0.07404255, 0.02485879, 0.03151626, 0.13495013,
        0.15786797, 0.15588643, 0.20923835, 0.08573047, 0.09344468]],
      dtype=float32)>

In [ ]:
## Copy weights from functional weights to custom model
subclassmodel.set_weights(functional_model.get_weights())

In [ ]:
assert len(functional_model.weights) == len(subclassmodel.weights)

In [ ]:
for a, b in zip(functional_model.weights, subclassmodel.weights):
    np.testing.assert_allclose(a.numpy(), b.numpy())

### 2.3 The case of stateless layers
Because stateless layers do not change the order or number of weights, models can have compatible architectures even if there are extra/missing stateless layers.





In [ ]:
inputs = keras.Input(shape=(784,), name="digits")
x = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = keras.layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = keras.layers.Dense(10, name="predictions")(x)
functional_model = keras.Model(inputs=inputs, outputs=outputs, name="3_layer_mlp")

inputs = keras.Input(shape=(784,), name="digits")
x = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = keras.layers.Dense(64, activation="relu", name="dense_2")(x)

# Add a dropout layer, which does not contain any weights.
x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(10, name="predictions")(x)
functional_model_with_dropout = keras.Model(
    inputs=inputs, outputs=outputs, name="3_layer_mlp"
)

functional_model_with_dropout.set_weights(functional_model.get_weights())

In [ ]:
for a, b in zip(functional_model.weights, functional_model_with_dropout.weights):
    np.testing.assert_allclose(a.numpy(), b.numpy())

## 3. Saving weights to disk and loading them back
Weights can be saved to disk by calling model.save_weights in the following formats:

1. TensorFlow Checkpoint
2. HDF5

The default format for model.save_weights is TensorFlow checkpoint. There are two ways to specify the save format:

1. **save_format** argument: Set the value to save_format="tf" or save_format="h5".
2. **path** argument: If the path ends with .h5 or .hdf5, then the HDF5 format is used. Other suffixes will result in a TensorFlow checkpoint unless save_format is set.


### 3.1 TensorFlow Checkpoint

In [ ]:
sequencial_model = keras.Sequential(
    [
        keras.Input(shape=(784, ), name='digits'),
        keras.layers.Dense(64, activation='relu', name='dense_1'),
        keras.layers.Dense(64, activation='relu', name='dense_2'),
        keras.layers.Dense(10, activation='softmax', name='prediction')
    ]
)

In [ ]:
sequencial_model.save_weights('ckpt')
load_status = sequencial_model.load_weights('ckpt')

In [ ]:
load_status.assert_consumed()


#### 3.1.1 Transfer Learning Example
Essentially, as long as two models have the same architecture, they are able to share the same checkpoint.



In [ ]:
inputs = keras.Input(shape=(784, ), name='digits')
x = keras.layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = keras.layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = keras.layers.Dense(10, activation='softmax', name='prediction')(x)
functional_model = keras.Model(inputs, outputs, name='3_layer_mlp')

In [ ]:
functional_model.summary()

Model: "3_layer_mlp"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, 784)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
prediction (Dense)           (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


#### 3.1.2 Extract a portion of functional model defined in the Setup section.
the follwing lines produce a new model that excludes the final output layer of the functional_model

In [ ]:
pretrained = keras.Model(
            functional_model.inputs, functional_model.layers[-1].input, name='pretrained_model')

In [ ]:
## Randomly assigned 'trained' weights.
for w in pretrained.weights:
    w.assign(tf.random.normal(w.shape))
pretrained.save_weights('pretrained_ckpt')

In [ ]:
pretrained.summary()

Model: "pretrained_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, 784)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
Total params: 54,400
Trainable params: 54,400
Non-trainable params: 0
_________________________________________________________________


In [ ]:
## Assume this is a seperate program where only pretrained_ckpt exists.
# Create a new functional model with a different output dimension
inputs = keras.Input(shape=(784,), name='digits')
x = keras.layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = keras.layers.Dense(64, activation='relu', name='dense_2')(x)
outputs=keras.layers.Dense(5, name='predictions')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name='new_digit_model')
model.summaryry()

Model: "new_digit_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, 784)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
predictions (Dense)          (None, 5)                 325       
Total params: 54,725
Trainable params: 54,725
Non-trainable params: 0
_________________________________________________________________


In [ ]:
## Load the weights from pretrained_ckpt into model
model.load_weights('pretrained_ckpt')

In [ ]:
## Check that all the pretrained weights have been loaded
for a, b in zip(pretrained.weights, model.weights):
    np.testing.assert_allclose(a.numpy(), b.numpy())

In [ ]:
### print("\n", "-" * 50)
model.summary()

Model: "new_digit_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, 784)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
predictions (Dense)          (None, 5)                 325       
Total params: 54,725
Trainable params: 54,725
Non-trainable params: 0
_________________________________________________________________


#### Sequential model
It is generally recommended to stick to the same API for building models. If you switch between Sequential and Functional, or Functional and subclassed, etc., then always rebuild the pre-trained model and load the pre-trained weights to that model.


In [ ]:
# Recreate the pretrained model, and load the saved weights.
inputs = keras.Input(shape=(784,), name="digits")
x = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = keras.layers.Dense(64, activation="relu", name="dense_2")(x)
pretrained_model = keras.Model(inputs=inputs, outputs=x, name="pretrained")
pretrained_model.summary()

Model: "pretrained"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, 784)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
Total params: 54,400
Trainable params: 54,400
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Sequential example:
model = keras.Sequential([pretrained_model, keras.layers.Dense(5, name="predictions")])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
pretrained (Functional)      (None, 64)                54400     
_________________________________________________________________
predictions (Dense)          (None, 5)                 325       
Total params: 54,725
Trainable params: 54,725
Non-trainable params: 0
_________________________________________________________________


In [ ]:
pretrained_model.load_weights("pretrained_ckpt")

# Warning! Calling `model.load_weights('pretrained_ckpt')` won't throw an error,
# but will *not* work as expected. If you inspect the weights, you'll see that
# none of the weights will have loaded. `pretrained_model.load_weights()` is the
# correct method to call.


In [ ]:
pretrained_model.weights

[<tf.Variable 'dense_1/kernel:0' shape=(784, 64) dtype=float32, numpy=
 array([[ 1.0397673 , -0.05201967, -0.745365  , ..., -0.18462151,
         -0.29110402,  1.1428523 ],
        [-0.29544547,  0.74784327, -0.24215454, ..., -0.61429846,
         -0.362189  ,  0.49086893],
        [-1.390071  , -0.0760475 , -0.21802928, ...,  0.17000744,
         -0.9297128 , -0.31069708],
        ...,
        [ 0.38178557, -1.5539563 , -1.2718143 , ...,  0.14016278,
          0.15168095,  1.1837689 ],
        [-0.33036786,  0.05328415, -0.8898189 , ..., -0.17068684,
          1.8069929 ,  0.1003458 ],
        [-1.2287658 ,  0.6008524 ,  1.3724065 , ...,  0.45753354,
          0.51558656,  0.70640534]], dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(64,) dtype=float32, numpy=
 array([ 0.55976397,  0.11353584,  0.99737406, -0.3814695 ,  1.1309181 ,
         0.22690475,  0.4245775 , -0.56957513,  0.34856254,  1.1837989 ,
        -0.3682391 ,  0.80762017,  0.2070313 , -1.6404922 ,  0.7442746 ,
  

### 3.2 HDF5 format                      
The HDF5 format contains weights grouped by layer names. The weights are lists ordered by concatenating the list of trainable weights to the list of non-trainable weights (same as layer.weights). Thus, a model can use a hdf5 checkpoint if it has the same layers and trainable statuses as saved in the checkpoint.



In [ ]:
# Runable Example
sequential_model = keras.Sequential(
    [
            keras.Input(shape=(784, ), name='digits'),
            keras.layers.Dense(64, activation='relu', name='dense_1'),
            keras.layers.Dense(64, activation='relu', name='dense_2'),
            keras.layers.Dense(10, name='predictions')
    ]
)
sequential_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
predictions (Dense)          (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


In [ ]:
sequencial_model.weights

[<tf.Variable 'dense_1/kernel:0' shape=(784, 64) dtype=float32, numpy=
 array([[ 0.03807154, -0.03411273, -0.01539298, ...,  0.0016689 ,
         -0.08054363, -0.01071978],
        [-0.08316234, -0.01622465, -0.03813309, ..., -0.065508  ,
          0.03159596, -0.03234336],
        [-0.02673952, -0.04008139,  0.0438318 , ...,  0.07462753,
         -0.01570794, -0.07352149],
        ...,
        [-0.04511328,  0.07932205, -0.02177209, ..., -0.08256465,
         -0.03875379, -0.04457112],
        [-0.00677938, -0.0766691 , -0.05910678, ..., -0.04280933,
         -0.04971677,  0.02203134],
        [ 0.05214521,  0.02506733,  0.04064305, ...,  0.06933377,
          0.03606524, -0.04857475]], dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(64,) dtype=float32, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [ ]:
sequential_model.save_weights('weights.h5')

In [ ]:
sequential_model.load_weights('weights.h5')

Note that changing layer.trainable may result in a different layer.weights ordering when the model contains nested layers.

In [ ]:
class NestedDenseLayer(keras.layers.Layer):
    def __init__(self, units, name=None):
        super(NestedDenseLayer, self).__init__(name=name)
        self.dense_1 = keras.layers.Dense(units, name="dense_1")
        self.dense_2 = keras.layers.Dense(units, name="dense_2")

    def call(self, inputs):
        return self.dense_2(self.dense_1(inputs))

In [ ]:
nested_model = keras.Sequential([keras.Input((784,)), NestedDenseLayer(10, "nested")])

In [ ]:
variable_names = [v.name for v in nested_model.weights]
print("variables: {}".format(variable_names))

variables: ['nested/dense_1/kernel:0', 'nested/dense_1/bias:0', 'nested/dense_2/kernel:0', 'nested/dense_2/bias:0']


In [ ]:
print("\nChanging trainable status of one of the nested layers...")
nested_model.get_layer("nested").dense_1.trainable = False



Changing trainable status of one of the nested layers...


In [ ]:
variable_names_2 = [v.name for v in nested_model.weights]
print("\nvariables: {}".format(variable_names_2))
print("variable ordering changed:", variable_names != variable_names_2)



variables: ['nested/dense_2/kernel:0', 'nested/dense_2/bias:0', 'nested/dense_1/kernel:0', 'nested/dense_1/bias:0']
variable ordering changed: True


#### Transfer learning example
When loading pretrained weights from HDF5, it is recommended to load the weights into the original checkpointed model, and then extract the desired weights/layers into a new model.



In [ ]:
def create_functional_model():
    inputs = keras.Input(shape=(784,), name="digits")
    x = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
    x = keras.layers.Dense(64, activation="relu", name="dense_2")(x)
    outputs = keras.layers.Dense(10, name="predictions")(x)
    return keras.Model(inputs=inputs, outputs=outputs, name="3_layer_mlp")


functional_model = create_functional_model()
functional_model.save_weights("pretrained_weights.h5")

# In a separate program:
pretrained_model = create_functional_model()
pretrained_model.load_weights("pretrained_weights.h5")

# Create a new model by extracting layers from the original model:
extracted_layers = pretrained_model.layers[:-1]
extracted_layers.append(keras.layers.Dense(5, name="dense_3"))
model = keras.Sequential(extracted_layers)
model.summary()


Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 325       
Total params: 54,725
Trainable params: 54,725
Non-trainable params: 0
_________________________________________________________________
